In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

# WGMS

## 1. GLACIER
General (and presumably static) information about each glacier

In [2]:
glacier = pd.read_csv("data/wgms/glacier.csv")

glacier_nz = glacier[glacier["POLITICAL_UNIT"] == "NZ"]

glacier_nz_drop = glacier_nz.drop(["POLITICAL_UNIT", "REMARKS", "GLACIER_REGION_CODE", "GLACIER_SUBREGION_CODE",
                                  "GEN_LOCATION", "SPEC_LOCATION", "PARENT_GLACIER",
                                  "PRIM_CLASSIFIC", "FORM", "FRONTAL_CHARS", "EXPOS_ACC_AREA", "EXPOS_ABL_AREA"], axis=1)
glacier_nz_drop.head()

C:\Users\Zach\AppData\Local\Temp\ipykernel_3492\3864064715.py:1: DtypeWarning: Columns (3,4,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  glacier = pd.read_csv("data/wgms/glacier.csv")


,NAME,WGMS_ID,LATITUDE,LONGITUDE
158448,ABEL,1546,-43.320000,170.630005
158449,ADAMS,2923,-43.320000,170.720001
158450,AILSA,2924,-44.786100,168.187000
158451,ALMER/SALISBURY,1548,-43.470001,170.220001
158452,ANDY,1590,-44.430000,168.369995


In [3]:
glacier_nz_drop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2869 entries, 158448 to 161316
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   NAME       2869 non-null   object 
 1   WGMS_ID    2869 non-null   int64  
 2   LATITUDE   2869 non-null   float64
 3   LONGITUDE  2869 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 112.1+ KB


## 2. CHANGE
Change in glacier thickness, area, and/or volume – typically from geodetic surveys



In [4]:
change = pd.read_csv("data/wgms/change.csv")

change_nz = change[change["POLITICAL_UNIT"] == "NZ"]

change_nz_drop = change_nz.drop(["POLITICAL_UNIT",
                                 "NAME",
                                 "SURVEY_ID",
                                 # "SURVEY_DATE",
                                 "REFERENCE_DATE",
                                 "LOWER_BOUND",
                                 "UPPER_BOUND",
                                 "AREA_CHANGE",  # all NaN
                                 "AREA_CHANGE_UNC",
                                 "AREA_SURVEY_YEAR",
                                 "THICKNESS_CHG_UNC",
                                 "VOLUME_CHANGE_UNC",
                                 "SD_PLATFORM_METHOD",
                                 "RD_PLATFORM_METHOD",
                                 "INVESTIGATOR",
                                 "SPONS_AGENCY",
                                 "REFERENCE",
                                 "REMARKS"], axis=1)
change_nz_drop.rename(columns = {'YEAR':'CHANGE_YEAR'}, inplace = True)
change_nz_drop.head()

,WGMS_ID,CHANGE_YEAR,SURVEY_DATE,THICKNESS_CHG,VOLUME_CHANGE
814867,2923,2013,20130228,5913.0,NaN
814868,2923,2018,20180307,-14433.0,NaN
814869,2923,2018,20180307,-13184.0,NaN
814870,2923,2014,20140224,-7845.0,NaN
814871,2923,2017,20170304,-7478.0,NaN


In [5]:
change_nz_drop["CHANGE_YEAR"].value_counts()

CHANGE_YEAR
2017    6998
2015    6584
2014    6162
2016    5848
2019    5516
2012    3764
2013    3107
2009    2853
2004    2758
2018     940
2011     720
2010     393
Name: count, dtype: int64

In [6]:
change_nz_drop[change_nz_drop['CHANGE_YEAR'] == 2009]["WGMS_ID"].nunique()

2767

In [7]:
change_nz_drop[change_nz_drop['CHANGE_YEAR'] == 2019]["WGMS_ID"].nunique()

2758

In [13]:
front_variation = pd.read_csv("data/wgms/front_variation.csv")

front_variation_nz = front_variation[front_variation["POLITICAL_UNIT"] == "NZ"]

front_variation_nz_drop = front_variation_nz.drop(["FRONT_VAR_UNC", 
                                                   "SURVEY_PLATFORM_METHOD", 
                                                   "INVESTIGATOR", 
                                                   "SPONS_AGENCY", 
                                                   "REFERENCE", 
                                                   "REMARKS",
                                                   "POLITICAL_UNIT",
                                                   "NAME"], axis=1)
front_variation_nz_drop.rename(columns = {'YEAR':'FRONT_VARIATION_YEAR'}, inplace = True)
front_variation_nz_drop.head()

,WGMS_ID,FRONT_VARIATION_YEAR,SURVEY_DATE,REFERENCE_DATE,FRONT_VARIATION,QUALITATIVE_VARIATION
41300,1546,1993,19930215,19890401.0,NaN,+X
41301,1546,1994,19940310,19930215.0,NaN,ST
41302,1546,1995,19950304,19940310.0,NaN,ST
41303,2923,1992,19920407,19870306.0,NaN,-X
41304,2923,1993,19930215,19920407.0,NaN,-X


In [14]:
tmp = front_variation_nz_drop[front_variation_nz_drop["QUALITATIVE_VARIATION"] == "ST"]
tmp["WGMS_ID"].nunique()

70

In [15]:
mass_balance = pd.read_csv("data/wgms/mass_balance.csv")

mass_balance_nz = mass_balance[mass_balance["POLITICAL_UNIT"] == "NZ"]

mass_balance_nz_drop = mass_balance_nz.drop(["LOWER_BOUND", 
                                             "UPPER_BOUND", 
                                             "WINTER_BALANCE_UNC", 
                                             "SUMMER_BALANCE_UNC", 
                                             "ANNUAL_BALANCE_UNC",
                                             "REMARKS",
                                             "POLITICAL_UNIT",
                                             "NAME"], axis=1)
mass_balance_nz_drop.rename(columns = {'YEAR':'MASS_BALANCE_YEAR', 'AREA':'MASS_BALANCE_YEAR_AREA'}, inplace = True)
mass_balance_nz_drop.head()

,WGMS_ID,MASS_BALANCE_YEAR,MASS_BALANCE_YEAR_AREA,WINTER_BALANCE,SUMMER_BALANCE,ANNUAL_BALANCE
44238,1597,2005,2.03,2875.0,-1499.0,1376.0
44239,1597,2006,2.03,2248.0,-1557.0,691.0
44240,1597,2007,2.03,3039.0,-2347.0,692.0
44241,1597,2008,2.03,2392.0,-4090.0,-1698.0
44242,1597,2009,2.03,1975.0,-2677.0,-702.0


In [16]:
special_event = pd.read_csv("data/wgms/special_event.csv")

special_event_nz = special_event[special_event["POLITICAL_UNIT"] == "NZ"]

special_event_nz_drop = special_event_nz.drop(["INVESTIGATOR", 
                                               "SPONS_AGENCY", 
                                               "REFERENCE", 
                                               "REMARKS",
                                               "POLITICAL_UNIT",
                                               "NAME",
                                               "EVENT_ID"], axis=1)
special_event_nz_drop.head()

,WGMS_ID,EVENT_DATE,ET_SURGE,ET_CALVING,ET_FLOOD,ET_AVALANCHE,ET_TECTONIC,ET_OTHER,EVENT_DESCRIPTION
2906,1580,19920502.0,False,False,False,False,False,True,"The rock avalanche, inspected on 5 May 1992, r..."
2907,1580,19920916.0,False,False,False,False,False,True,"The rock avalanche, inspected on 20 September ..."
2908,1074,19911214.0,False,False,False,False,True,True,Mount Cook Rock Avalanche occurred on 14 Decem...
2909,1074,19949999.0,False,False,False,False,False,True,"During a storm of January 1994, the river brea..."


In [17]:
reconstruction_front_variation = pd.read_csv("data/wgms/reconstruction_front_variation.csv")

reconstruction_front_variation_nz = reconstruction_front_variation[reconstruction_front_variation["POLITICAL_UNIT"] == "NZ"]

reconstruction_front_variation_nz_drop = reconstruction_front_variation_nz.drop(["YEAR_UNC", 
                                                                                 "REF_YEAR_UNC", 
                                                                                 "FRONT_VAR_POS_UNC", 
                                                                                 "FRONT_VAR_NEG_UNC",
                                                                                 "ELEVATION_UNC", 
                                                                                 "METHOD_CODE", 
                                                                                 "METHOD_REMARKS", 
                                                                                 "REMARKS", 
                                                                                 "QUALITATIVE_VARIATION",  # all NaN
                                                                                 "LOWEST_ELEVATION", 
                                                                                 "HIGHEST_ELEVATION",
                                                                                 "MORAINE_DEFINED_MAX",
                                                                                 "POLITICAL_UNIT",
                                                                                 "NAME"], axis=1)
reconstruction_front_variation_nz_drop.rename(columns = {'YEAR':'RECONSTRUCTION_FRONT_VARIATION_YEAR'}, inplace = True)
reconstruction_front_variation_nz_drop.head()

,WGMS_ID,REC_SERIES_ID,RECONSTRUCTION_FRONT_VARIATION_YEAR,REFERENCE_YEAR,FRONT_VARIATION
1874,899,36,1780,1600.0,-560.0
1875,899,36,1820,1780.0,141.0
1876,899,36,1865,1820.0,-240.0
1877,899,36,1867,1865.0,-21.0
1878,899,36,1886,1867.0,-29.0


## 3. Merge WGMS
change and glacier

In [8]:
pd.set_option('display.max_columns', None)

In [10]:
merge_1 = pd.merge(change_nz_drop, glacier_nz_drop, how="left", on="WGMS_ID")
merge_1.head()

,WGMS_ID,CHANGE_YEAR,SURVEY_DATE,THICKNESS_CHG,VOLUME_CHANGE,NAME,LATITUDE,LONGITUDE
0,2923,2013,20130228,5913.0,NaN,ADAMS,-43.32,170.720001
1,2923,2018,20180307,-14433.0,NaN,ADAMS,-43.32,170.720001
2,2923,2018,20180307,-13184.0,NaN,ADAMS,-43.32,170.720001
3,2923,2014,20140224,-7845.0,NaN,ADAMS,-43.32,170.720001
4,2923,2017,20170304,-7478.0,NaN,ADAMS,-43.32,170.720001


In [11]:
merge_1.rename(columns={'THICKNESS_CHG': 'THICKNESS_CHANGE'}, inplace=True)

In [13]:
merge_1 = merge_1.iloc[:,[0,5,1,2,3,4,6,7]]

In [14]:
merge_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45643 entries, 0 to 45642
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   WGMS_ID           45643 non-null  int64  
 1   NAME              45643 non-null  object 
 2   CHANGE_YEAR       45643 non-null  int64  
 3   SURVEY_DATE       45643 non-null  int64  
 4   THICKNESS_CHANGE  45643 non-null  float64
 5   VOLUME_CHANGE     13790 non-null  float64
 6   LATITUDE          45643 non-null  float64
 7   LONGITUDE         45643 non-null  float64
dtypes: float64(4), int64(3), object(1)
memory usage: 2.8+ MB


In [15]:
merge_1.head()

,WGMS_ID,NAME,CHANGE_YEAR,SURVEY_DATE,THICKNESS_CHANGE,VOLUME_CHANGE,LATITUDE,LONGITUDE
0,2923,ADAMS,2013,20130228,5913.0,NaN,-43.32,170.720001
1,2923,ADAMS,2018,20180307,-14433.0,NaN,-43.32,170.720001
2,2923,ADAMS,2018,20180307,-13184.0,NaN,-43.32,170.720001
3,2923,ADAMS,2014,20140224,-7845.0,NaN,-43.32,170.720001
4,2923,ADAMS,2017,20170304,-7478.0,NaN,-43.32,170.720001


# GLIMS

In [16]:
gla_glims = gpd.read_file('data/glims/nz_glaciers_polygons.shp')

In [17]:
gla_glims.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9032 entries, 0 to 9031
Data columns (total 39 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   line_type   9032 non-null   object  
 1   anlys_id    9032 non-null   float64 
 2   glac_id     9032 non-null   object  
 3   anlys_time  9032 non-null   object  
 4   area        9032 non-null   float64 
 5   db_area     9032 non-null   float64 
 6   width       9032 non-null   float64 
 7   length      9032 non-null   float64 
 8   primeclass  9032 non-null   float64 
 9   min_elev    9032 non-null   float64 
 10  mean_elev   9032 non-null   float64 
 11  max_elev    9032 non-null   float64 
 12  src_date    9032 non-null   object  
 13  rec_status  9032 non-null   object  
 14  glac_name   9032 non-null   object  
 15  wgms_id     9032 non-null   object  
 16  local_id    9032 non-null   object  
 17  glac_stat   9032 non-null   object  
 18  gone_date   9032 non-null   object  
 19

In [18]:
gla_glims["src_date"].value_counts()

src_date
2019-03-28T00:00:00    4077
2016-03-11T00:00:00    3950
2009-02-17T22:38:29     604
1600-09-09T00:00:00     401
Name: count, dtype: int64

In [19]:
columns_to_drop = [
    'wgms_id', 'local_id', 'glac_stat', 'gone_date', 'gone_dt_e',
    'subm_id', 'rc_id', 'chief_affl', 'conn_lvl', 'surge_type',
    'term_type', 'gtng_o1reg', 'gtng_o2reg', 'rgi_gl_typ', 'loc_unc_x',
    'loc_unc_y', 'glob_unc_x', 'glob_unc_y', 'submitters', 'analysts',
    'mean_elev', 'max_elev', 'width', 'length', 'primeclass',
    'rec_status', 'proc_desc',
    'anlys_id', 'anlys_time', 'glac_name', 'release_dt', 'geog_area',
    'line_type', 'area', 'db_area'
]
gla_glims_cleaned = gla_glims.drop(columns=columns_to_drop)
gla_glims_cleaned

,glac_id,min_elev,src_date,geometry
0,G170910E43118S,0.0,1600-09-09T00:00:00,"POLYGON Z ((170.91298 -43.11761 0.00000, 170.9..."
1,G172719E42080S,0.0,1600-09-09T00:00:00,"POLYGON Z ((172.72521 -42.07704 0.00000, 172.7..."
2,G169791E43889S,0.0,1600-09-09T00:00:00,"POLYGON Z ((169.78505 -43.88165 0.00000, 169.7..."
3,G168618E44432S,0.0,1600-09-09T00:00:00,"POLYGON Z ((168.62173 -44.43379 0.00000, 168.6..."
4,G170012E43673S,0.0,2009-02-17T22:38:29,"POLYGON Z ((170.00462 -43.66741 0.00000, 170.0..."
...,...,...,...,...
9027,G171135E43078S,0.0,1600-09-09T00:00:00,"POLYGON Z ((171.13377 -43.07421 0.00000, 171.1..."
9028,G170930E43172S,0.0,1600-09-09T00:00:00,"POLYGON Z ((170.93579 -43.18184 0.00000, 170.9..."
9029,G169768E43774S,0.0,1600-09-09T00:00:00,"POLYGON Z ((169.75320 -43.77087 0.00000, 169.7..."
9030,G168427E44498S,0.0,1600-09-09T00:00:00,"POLYGON Z ((168.43220 -44.49552 0.00000, 168.4..."


In [20]:
gla_glims_cleaned.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9032 entries, 0 to 9031
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   glac_id   9032 non-null   object  
 1   min_elev  9032 non-null   float64 
 2   src_date  9032 non-null   object  
 3   geometry  9032 non-null   geometry
dtypes: float64(1), geometry(1), object(2)
memory usage: 282.4+ KB


In [21]:
gla_glims_cleaned['min_elev'].value_counts()

min_elev
0.0       5082
799.0      107
594.0       36
1080.0      29
917.0       25
          ... 
1672.0       1
1405.0       1
1269.0       1
1712.0       1
2203.0       1
Name: count, Length: 1051, dtype: int64

In [22]:
gla_glims_cleaned.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [23]:
# gla_glims_cleaned["year"] = gla_glims_cleaned['src_date'].str[:4].astype(int)
# gla_glims_cleaned["month"] = gla_glims_cleaned['src_date'].str[5:7].astype(int)
# gla_glims_cleaned["day"] = gla_glims_cleaned['src_date'].str[8:10].astype(int)
# gla_glims_cleaned = gla_glims_cleaned.drop(["src_date"], axis=1)
# gla_glims_cleaned.head()

# Spatial Join

In [24]:
merge_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45643 entries, 0 to 45642
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   WGMS_ID           45643 non-null  int64  
 1   NAME              45643 non-null  object 
 2   CHANGE_YEAR       45643 non-null  int64  
 3   SURVEY_DATE       45643 non-null  int64  
 4   THICKNESS_CHANGE  45643 non-null  float64
 5   VOLUME_CHANGE     13790 non-null  float64
 6   LATITUDE          45643 non-null  float64
 7   LONGITUDE         45643 non-null  float64
dtypes: float64(4), int64(3), object(1)
memory usage: 2.8+ MB


In [25]:
gdf_wgms = (
    gpd.GeoDataFrame(
        merge_1, geometry=gpd.points_from_xy(merge_1.LONGITUDE, merge_1.LATITUDE), crs="EPSG:4326"
    )
)

gdf_wgms.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 45643 entries, 0 to 45642
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   WGMS_ID           45643 non-null  int64   
 1   NAME              45643 non-null  object  
 2   CHANGE_YEAR       45643 non-null  int64   
 3   SURVEY_DATE       45643 non-null  int64   
 4   THICKNESS_CHANGE  45643 non-null  float64 
 5   VOLUME_CHANGE     13790 non-null  float64 
 6   LATITUDE          45643 non-null  float64 
 7   LONGITUDE         45643 non-null  float64 
 8   geometry          45643 non-null  geometry
dtypes: float64(4), geometry(1), int64(3), object(1)
memory usage: 3.1+ MB


In [26]:
gdf_wgms.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [38]:
# wgms is points, glims is polygon, so based on glims, put wgms on
wgms_glims =  gla_glims_cleaned.sjoin(gdf_wgms, predicate="intersects")

In [39]:
# Drop volume change due to too many missing data
wgms_glims = wgms_glims.drop(['index_right', 'VOLUME_CHANGE'], axis=1)
wgms_glims = wgms_glims.reset_index(drop=True)

In [51]:
wgms_glims_no_duplicate = wgms_glims.drop_duplicates()

In [54]:
wgms_glims_2004 = wgms_glims_no_duplicate[wgms_glims_no_duplicate['CHANGE_YEAR'] == 2004]

In [65]:
len(wgms_glims_2004)

2826

In [69]:
wgms_glims_2004.to_file("data_cleaned/wgms_glims_2004.gpkg")

In [56]:
wgms_glims_2009 = wgms_glims_no_duplicate[wgms_glims_no_duplicate['CHANGE_YEAR'] == 2009]

In [66]:
len(wgms_glims_2009)

3001

In [70]:
wgms_glims_2009.to_file("data_cleaned/wgms_glims_2009.gpkg")

In [58]:
wgms_glims_2014 = wgms_glims_no_duplicate[wgms_glims_no_duplicate['CHANGE_YEAR'] == 2014]

In [67]:
len(wgms_glims_2014)

11109

In [71]:
wgms_glims_2014.to_file("data_cleaned/wgms_glims_2014.gpkg")

In [59]:
wgms_glims_2019 = wgms_glims_no_duplicate[wgms_glims_no_duplicate['CHANGE_YEAR'] == 2019]

In [68]:
len(wgms_glims_2019)

5649

In [72]:
wgms_glims_2019.to_file("data_cleaned/wgms_glims_2019.gpkg")